In [34]:
import os
import numpy as np
import pandas as pd
import torch
import torchvision.transforms as tr
from torch.utils import data
from torch.optim import lr_scheduler
import FocalLoss
import torch.optim as optim
from torch.autograd import Variable
import torch.nn as nn

In [55]:
class Rain_Loader(data.Dataset):

    def __init__(self, data_path, train):
        self.kind = ['No', 'Yes']
        self.file = pd.read_csv(data_path)
        self.train = train
        
        if self.train:
            self.labels = self.file['Attribute23']
            self.file.drop(columns=['Attribute23'], inplace=True)
        
        self.data = self.file.values
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self,index):

        data = self.data[index]

        if self.train:
            label = self.labels[index]
        else:
            label = 0
        return data, label    

In [72]:
class MyNet(nn.Module):
    def __init__(self, num_classes=2):
        super(MyNet, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(22, 11),
            nn.ReLU(),
            nn.Linear(11, 6),
            nn.ReLU(),
            nn.Linear(6, num_classes),
        )
    def forward(self, x):
        x = x.view(1, len(x)*len(x[0]))
        x = self.classifier(x)
        return x
    
Net = MyNet(num_classes=2)
Net.train()

MyNet(
  (classifier): Sequential(
    (0): Linear(in_features=22, out_features=11, bias=True)
    (1): ReLU()
    (2): Linear(in_features=11, out_features=6, bias=True)
    (3): ReLU()
    (4): Linear(in_features=6, out_features=2, bias=True)
  )
)

In [83]:
# My dataLoader
train_data = Rain_Loader(data_path = 'train2.csv',train = True)
train_dataloader = data.DataLoader(dataset=train_data,batch_size = 1, shuffle=True)

test_data = Rain_Loader(data_path = 'test2.csv',train = False)
test_dataloader = data.DataLoader(dataset=test_data,batch_size = 1, shuffle=False)

# My Model
model = MyNet(num_classes = 2)
model = model.float()
model.train()

# Loss function
criterion1 = FocalLoss.FocalLoss(gamma=5.5, alpha=0.16, size_average=False)
# optimizer
optimizer = optim.Adam(model.parameters(), lr=0.01)

# learning rate schedular
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [84]:
iter_time = 0
run_loss = 0.0
times = 15

for epoch in range(times):
    print("Epoch [",epoch+1," / " , times , "]", end='')
    # Start Training

    for i,Data in enumerate(train_dataloader,start = 1):

        iter_time+=1
        # Load Data
        row,labels = Data
        row,labels = Variable(row) ,Variable(labels)
        
        # Feeding into model
        preds = model(row.float())
        # Get loss
        labels = labels.to(dtype=torch.int64)
        loss = criterion1(preds, labels)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        run_loss += loss.item()
        
    # schedule my learning rate
    exp_lr_scheduler.step()
    print(", Loss : %.4f" %((run_loss / len(train_dataloader))))
    run_loss = 0
print('finish')

Epoch [ 1  /  15 ], Loss : 0.0037
Epoch [ 2  /  15 ], Loss : 0.0034
Epoch [ 3  /  15 ], Loss : 0.0038
Epoch [ 4  /  15 ], Loss : 0.0034
Epoch [ 5  /  15 ], Loss : 0.0033
Epoch [ 6  /  15 ], Loss : 0.0033
Epoch [ 7  /  15 ], Loss : 0.0033
Epoch [ 8  /  15 ], Loss : 0.0033
Epoch [ 9  /  15 ], Loss : 0.0032
Epoch [ 10  /  15 ], Loss : 0.0033
Epoch [ 11  /  15 ], Loss : 0.0032
Epoch [ 12  /  15 ], Loss : 0.0032
Epoch [ 13  /  15 ], Loss : 0.0032
Epoch [ 14  /  15 ]

KeyboardInterrupt: 

In [85]:
ans = pd.DataFrame(columns=['id', 'ans'])
for i,Data in enumerate(test_dataloader):
    
    row,labels = Data
    row,labels = Variable(row) ,Variable(labels)
    
    preds = model(row.float())
    
    if(i % 50 == 0):
        s = nn.Softmax()
        preds = s(preds)
        print(i, preds)
        
    _, preds = torch.max(preds.data, 1)
    
    ans.loc[i] = [str(i) + '.0', preds.item()]
ans.to_csv('ans.csv',index=None, quoting=2)

D:\Program Files (x86)\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  # This is added back by InteractiveShellApp.init_path()


0 tensor([[0.5727, 0.4273]], grad_fn=<SoftmaxBackward>)
50 tensor([[0.5727, 0.4273]], grad_fn=<SoftmaxBackward>)
100 tensor([[0.4935, 0.5065]], grad_fn=<SoftmaxBackward>)
150 tensor([[0.5727, 0.4273]], grad_fn=<SoftmaxBackward>)
200 tensor([[0.5314, 0.4686]], grad_fn=<SoftmaxBackward>)
250 tensor([[0.4767, 0.5233]], grad_fn=<SoftmaxBackward>)
300 tensor([[0.5727, 0.4273]], grad_fn=<SoftmaxBackward>)
350 tensor([[0.5727, 0.4273]], grad_fn=<SoftmaxBackward>)
400 tensor([[0.4606, 0.5394]], grad_fn=<SoftmaxBackward>)
450 tensor([[0.5727, 0.4273]], grad_fn=<SoftmaxBackward>)
500 tensor([[0.4453, 0.5547]], grad_fn=<SoftmaxBackward>)
550 tensor([[0.4480, 0.5520]], grad_fn=<SoftmaxBackward>)
600 tensor([[0.5455, 0.4545]], grad_fn=<SoftmaxBackward>)
650 tensor([[0.5727, 0.4273]], grad_fn=<SoftmaxBackward>)
700 tensor([[0.4628, 0.5372]], grad_fn=<SoftmaxBackward>)
750 tensor([[0.4602, 0.5398]], grad_fn=<SoftmaxBackward>)
800 tensor([[0.4313, 0.5687]], grad_fn=<SoftmaxBackward>)


In [21]:
ans.describe()

,id,ans
count,824,824
unique,824,2
top,210.0,1
freq,1,450
